# Capstone Project: **SchedMate**

Import semua library yang dibutuhkan

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
import matplotlib.image as mpimg
import os
import datetime as dt
import tensorflow as tf
import keras
import scipy
import sklearn
import json
import re
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix as cm
from google.colab import files
from joblib import dump

### GATHERING DATA

Pada tahap ini dilakukan pembacaan dataset yang akan dianalisis

In [ ]:
dataset = pd.read_csv("/content/Dataset Schedmate - Dataset Schedmate.csv")

In [ ]:
dataset.head()

,ulangan,pts_pas,tugas,presentasi,proyek,ekstrakurikuler,organisasi,agama,pkn,olahraga,...,biologi,fisika,kimia,senbud,prakarya,sejarah,ekonomi,sosiologi,geografi,komputer
0,5,5,3,4,5,3,3,2,3,2,...,3,5,4,1,1,4,1,2,4,2
1,5,5,4,3,3,1,1,2,3,1,...,4,4,4,1,1,2,3,3,5,3
2,4,4,4,4,4,2,1,1,1,1,...,3,4,4,1,2,2,2,2,4,4
3,3,5,5,5,5,3,3,2,2,2,...,2,3,2,1,2,3,2,4,4,3
4,4,3,4,5,5,1,2,1,1,1,...,2,5,1,1,3,3,2,2,3,2


### ASSESSING DATA

Melalukan penilaian terhadap dataset yang telah dibaca sebelumnya. Pada proses ini juga data akan diperiksa kelengkapannya seperti missing value, duplikasi, dan lainnya.

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602 entries, 0 to 601
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   ulangan          602 non-null    int64
 1   pts_pas          602 non-null    int64
 2   tugas            602 non-null    int64
 3   presentasi       602 non-null    int64
 4   proyek           602 non-null    int64
 5   ekstrakurikuler  602 non-null    int64
 6   organisasi       602 non-null    int64
 7   agama            602 non-null    int64
 8   pkn              602 non-null    int64
 9   olahraga         602 non-null    int64
 10  indonesia        602 non-null    int64
 11  inggris          602 non-null    int64
 12  mandarin         602 non-null    int64
 13  matematika       602 non-null    int64
 14  biologi          602 non-null    int64
 15  fisika           602 non-null    int64
 16  kimia            602 non-null    int64
 17  senbud           602 non-null    int64
 18  prakarya  

Periksa apakah terdapat duplikasi data

In [ ]:
print("Jumlah duplikasi: ", dataset.duplicated().sum())

Jumlah duplikasi:  0


Periksa apakah terdapat missing value pada data

In [ ]:
dataset.isna().sum()

,0
ulangan,0
pts_pas,0
tugas,0
presentasi,0
proyek,0
ekstrakurikuler,0
organisasi,0
agama,0
pkn,0
olahraga,0


### CLEANING DATA

Sudah tidak terdapat duplikasi, missing value, dan lainnya pada data


### PEMODELAN DATA

Penentuan bobot untuk jenis tugas dan mata pelajaran, bobot aktivitas di buat lebih besar dibandiing dengan bobot mata pelajaran

In [ ]:
w_activity = 0.5
w_subject = 0.5

Hitung skor prioritas yang menggabungkan bobot aktivitas dan bobot mata pelajaran

In [ ]:
dataset['priority_Score'] = (
    w_activity * dataset[['ulangan', 'pts_pas', 'tugas', 'presentasi', 'proyek', 'ekstrakurikuler', 'organisasi']].max(axis=1) +
    w_subject * dataset[['agama', 'pkn', 'olahraga', 'indonesia', 'inggris', 'mandarin', 'matematika', 'biologi', 'fisika', 'kimia', 'senbud', 'prakarya', 'sejarah', 'ekonomi', 'sosiologi', 'geografi', 'komputer']].max(axis=1)
)

Melihat priority score yang sudah dihitung

In [ ]:
print(dataset[['priority_Score']].head())
print(dataset['priority_Score'].describe())

   priority_Score
0             5.0
1             5.0
2             4.5
3             4.5
4             5.0
count    602.000000
mean       4.787375
std        0.296356
min        3.500000
25%        4.500000
50%        5.000000
75%        5.000000
max        5.000000
Name: priority_Score, dtype: float64


Tambahkan beberapa data secara manual untuk memperbesar dataset

In [ ]:
data_add = pd.DataFrame({
    'ulangan': [1, 1],
    'pts_pas': [1, 2],
    'tugas': [1, 1],
    'presentasi': [1, 2],
    'proyek': [1, 1],
    'ekstrakurikuler': [1, 2],
    'organisasi': [1, 1],
    'agama': [1, 2],
    'pkn': [1, 1],
    'olahraga': [1, 1],
    'indonesia': [1, 2],
    'inggris': [1, 1],
    'mandarin': [1, 1],
    'matematika': [1, 1],
    'biologi': [1, 2],
    'fisika': [1, 1],
    'kimia': [1, 1],
    'senbud': [1, 1],
    'prakarya': [1, 1],
    'sejarah': [1, 1],
    'ekonomi': [1, 1],
    'sosiologi': [1, 1],
    'geografi': [1, 1],
    'komputer': [1, 1],
    'priority_Score': [2, 3],
    'recommendation': [1, 2]
})


Menggabungkan dataset

In [ ]:
dataset = pd.concat([dataset, data_add], ignore_index=True)

Kategorikan priority score menjadi lima kategori kelas

In [ ]:
bins = [0, 4, 6, 8, 10, 12]
labels = [1, 2, 3, 4, 5]
dataset['recommendation'] = pd.cut(dataset['priority_Score'], bins=bins, labels=labels).astype(int)

In [ ]:
print(dataset['recommendation'].value_counts())

recommendation
2    574
1     30
Name: count, dtype: int64


Memisahkan X (fitur) yang merupakan semua kolom kecuali target dan skor prioritas dan Y (target) yang merupakan kolom kategori prioritas

In [ ]:
X = dataset.drop(columns=['priority_Score'])
y = dataset['priority_Score']

Membagi dataset menjadi 80% data test dan 20% data train, kemudian menangani ketidakseimbangan data dengan SMOTE (Synthetic Minority Oversampling Technique)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Lakukan normalisasi data dengan menggunakan StandarScaler

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Modeling dengan menggunakan TensorFlow

Model akhir: Membuat lima kategori berdasarkan prioritas

In [ ]:
model = Sequential([
    Dense(128, kernel_regularizer='l2', input_shape=(X_train_scaled.shape[1],)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64, kernel_regularizer='l2'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.1),
    Dense(1, activation='linear')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Compile model dengan menggunakan optimizer Adam, categorical crossentropy dan metricsnya berupa accuracy

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])


Optimasi menggunakan callbacks

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

Melatih model menggunakan data yang sudah diproses sebelumnya

In [ ]:
history = model.fit(X_train_scaled, y_train,
                    validation_data=(X_test_scaled, y_test),
                    epochs=200,
                    batch_size=16,
                    verbose=1,
                    callbacks=[early_stopping, lr_scheduler])

Epoch 1/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - loss: 24.7982 - mae: 4.7004 - val_loss: 22.1076 - val_mae: 4.5512 - learning_rate: 0.0010
Epoch 2/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 21.0986 - mae: 4.3573 - val_loss: 18.0166 - val_mae: 4.0780 - learning_rate: 0.0010
Epoch 3/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 17.5819 - mae: 3.9323 - val_loss: 14.6674 - val_mae: 3.6432 - learning_rate: 0.0010
Epoch 4/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 15.0084 - mae: 3.5931 - val_loss: 11.3968 - val_mae: 3.1632 - learning_rate: 0.0010
Epoch 5/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 11.5584 - mae: 3.0652 - val_loss: 8.0721 - val_mae: 2.5740 - learning_rate: 0.0010
Epoch 6/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.4273 - mae: 2.4649 - val_loss: 5.5079 - val_mae: 2.0168 - learning_rate: 0.0010
Epoch 7/200
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.5270 - mae: 1.8432 - val_loss: 3.3743 - val_mae: 1.3797 - learning_rate: 0.0010
Epo

Menyimpan scaler dalam bentuk joblib dan model dalam bentuk keras

In [ ]:
dump(scaler, 'scaler.joblib')
model.save('model_schedmate.keras')

Mengunduh file model dalam keras

In [ ]:
files.download('model_schedmate.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Mengunduh scaler

In [ ]:
files.download('scaler.joblib')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Melakukan prediksi pada data uji yang telah diskalakan serta memeriksa nilai NaN dalam y_pred dan menggantinya dengan nilai yang sesuai

In [ ]:
y_pred = model.predict(X_test_scaled)
y_pred = np.nan_to_num(y_pred)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


Menghitung metrik evaluasi

In [ ]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"MSE: {mse:.4f}, MAE: {mae:.4f}")

MSE: 0.0571, MAE: 0.2007
